In [ ]:
# This did NOT work, the text had way too many new lines in it. Removing them all gave me one giant sentence
# with no spaces

from PyPDF2 import PdfFileReader
import re

reader = PdfFileReader("textrank algorithm paper.pdf")
num_pages = reader.numPages
page = reader.pages[0]
re.sub("\n", "", page.extractText())

In [ ]:
# This had 
import pdfplumber

with pdfplumber.open("textrank algorithm paper.pdf") as pdf:
    page = pdf.pages[0]
    left = page.crop((0, 0, 0.5 * page.width, 0.9 * page.height))
    right = page.crop((0.5 * page.width, 0, page.width, page.height))
    l_text = left.extract_text()
    r_text = right.extract_text()
    text = l_text + " " + r_text
    print(text)

Code below was obtained from [here](https://pdfminersix.readthedocs.io/en/latest/tutorial/composable.html)

In [6]:
from io import StringIO

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert_pdf_to_string(file_path):
    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return(output_string.getvalue())

file_path = ''  # !
text = convert_pdf_to_string("textrank algorithm paper.pdf")
print(text)

TextRank: Bringing Order into Texts

Rada Mihalcea and Paul Tarau
Department of Computer Science
University of North Texas
rada,tarau (cid:1) @cs.unt.edu

Abstract

In this paper, we introduce TextRank – a graph-based
ranking model for text processing, and show how this
model can be successfully used in natural language
applications. In particular, we propose two innova-
tive unsupervised methods for keyword and sentence
extraction, and show that the results obtained com-
pare favorably with previously published results on
established benchmarks.

1 Introduction

Graph-based ranking algorithms like Kleinberg’s
HITS algorithm (Kleinberg, 1999) or Google’s
PageRank (Brin and Page, 1998) have been success-
fully used in citation analysis, social networks, and
the analysis of the link-structure of the World Wide
Web. Arguably, these algorithms can be singled out
as key elements of the paradigm-shift triggered in
the ﬁeld of Web search technology, by providing a
Web page ranking mechanism t